In [18]:
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Load your dataset (replace 'final_scraped_data.csv' with your actual file)
df = pd.read_csv("final_scraped_data.csv")

# Keep the 'Object Number' column as well
df = df[['Object Number', 'Description']]

# Convert the 'Description' column to strings (handle NaN values as 'nan')
df['Description'] = df['Description'].astype(str)

# Remove rows where 'Description' column is null (NaN or None)
df = df[df['Description'] != 'nan']

# df = df.head(1000)

# Function to extract relevant entities from the description
def extract_entities(description):
    # Process the description with spaCy
    doc = nlp(description)
    
    # Initialize a dictionary to store extracted entities (set to None if not found)
    entities = {
        'Culture Area d': None,
        'Culture d': None,
        'Date Made d': None,
        'Iconography d': None,
        'Materials d': None,
        'Period d': None,
        'Provenience d': None,
        'Technique d': None
    }
    
    # Extract entities
    for ent in doc.ents:
        if ent.label_ == 'MATERIAL':  # Materials
            entities['Materials d'] = ent.text
        elif ent.label_ == 'CULTURE':  # Culture
            entities['Culture d'] = ent.text
        elif ent.label_ == 'DATE' or ent.label_ == 'CARDINAL':  # Date Made / Period
            entities['Date Made d'] = ent.text
        elif ent.label_ == 'ORG':  # Iconography (assuming some organization references)
            entities['Iconography d'] = ent.text
        elif ent.label_ == 'GPE':  # Provenience (Geopolitical locations)
            entities['Provenience d'] = ent.text
        elif ent.label_ == 'TIME':  # Period (time reference)
            entities['Period d'] = ent.text
        # Optionally, you can extend other conditions as needed, e.g., for 'Technique'
    
    return entities

# Apply the function to the 'Description' column
df['Entities'] = df['Description'].apply(extract_entities)

# Expand the 'Entities' dictionary into separate columns
for col in df['Entities'][0].keys():
    df[col] = df['Entities'].apply(lambda x: x[col])

# Optionally drop the 'Entities' column as it is now expanded
df.drop(columns=['Entities'], inplace=True)

# Display the updated DataFrame for the first 5 rows
print(df.head(5))  # Display first 5 rows

# Save the updated DataFrame to a new CSV file
df.to_csv("processed_data_with_object_numbers.csv", index=False)

  Object Number                                        Description  \
0         10005           Showing artificial and natural fracture.   
3         10115  The fragment of pottery is unusually thick, an...   
4         10121                                         Spade like   
5         10126                                               Thin   
6         10230                                              Split   

  Culture Area d Culture d Date Made d Iconography d Materials d Period d  \
0           None      None        None          None        None     None   
3           None      None        None          None        None     None   
4           None      None        None          None        None     None   
5           None      None        None          None        None     None   
6           None      None        None          None        None     None   

  Provenience d Technique d  
0          None        None  
3     Tennessee        None  
4          None        Non

In [9]:
null_values = df.isna().sum()

# Display the number of null values for each column
print(null_values)

Object Number         0
Description           0
Culture Area d    21212
Culture d         21212
Date Made d       13028
Iconography d     15502
Materials d       21212
Period d          21189
Provenience d     18137
Technique d       21212
dtype: int64


In [11]:
df.drop(columns=['Culture Area d', 'Culture d', 'Materials d', 'Period d', 'Provenience d', 'Technique d'], inplace=True)

In [12]:
null_values = df.isna().sum()

# Display the number of null values for each column
print(null_values)

Object Number        0
Description          0
Date Made d      13028
Iconography d    15502
dtype: int64


In [13]:
df2 = pd.read_csv("midCleaning.csv")

In [14]:
df2 = pd.concat([df2, df], axis=1)

In [15]:
df2.columns

Index(['Object Number', 'image_path', 'Culture Area', 'Culture', 'Date Made',
       'Description', 'Iconography', 'Materials', 'Object Title', 'Period',
       'Provenience', 'Section', 'Technique', 'Object Number', 'Description',
       'Date Made d', 'Iconography d'],
      dtype='object')

In [16]:
df2.to_csv("added columns with nlp.csv", index=False)

In [17]:
# Check if all values in the row are not null in df2
non_null_rows = df2.notna().all(axis=1)

# Get the rows where nothing is null
rows_with_no_nulls = df2[non_null_rows]

# Display the number of rows where nothing is null
print(f"Number of rows where nothing is null: {len(rows_with_no_nulls)}")

Number of rows where nothing is null: 6
